# Anime recommendations system

![Image](https://www.geekjunior.fr/wp-content/uploads/2021/01/538943.jpg)

**Le travail effectué dans ce notebook se base sur le travail réalisé par [Yonatan Rabinovich](https://www.kaggle.com/yonatanrabinovich) sur [ce notebook](https://www.kaggle.com/yonatanrabinovich/anime-recommendations-project), ainsi que celui de [GeekyDNP](https://www.kaggle.com/dipayanpal) sur [ce second notebook](https://www.kaggle.com/dipayanpal/anime-recommendation)**

In [ ]:
# On commence par importer nos librairies
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy as sp

%matplotlib inline

print("Init.")

## Présentation du dataset

In [ ]:
df_anime = pd.read_csv("../input/anime-recommendations-database/anime.csv")
df_rating = pd.read_csv("../input/anime-recommendations-database/rating.csv")

In [ ]:
# Nombre d'entrées
df_anime.shape[0]

In [ ]:
# Nombre d'entrées
df_rating.shape[0]

In [ ]:
# Nombre d'utilisateurs uniques
df_rating['user_id'].nunique()

Le dataset se compose de deux jeux de données :

- Un CSV de 12294 animes
- Un CSV contentenant les notes de 73515 utilisateurs

Les données ont étées recueillies sur [MyAnimeList.net](https://myanimelist.net/apiconfig/references/api/v2) à l'aide de leur API. Sur ce site, chaque utilisateur peut ajouter des animes à leur liste de visionnage ainsi que de leur attribuer (ou non) une note allant de 1 à 10. Le dataset se compose de la collecte de ces données.

### Objectifs du notebook

Ce notebook a pour objectif d'explorer et de comprendre à partir de notebooks existants une façons de créer des recommandations pour un utilisateur en fonction de la similarité entre deux animes.

Dans un second temps, nous chercherons à déterminer la note moyenne d'un anime en fonction de leur similarité. 

## Nos jeux de données

 ### Anime Dataset
 Ce dataset appelé **anime** contient **12294** entrées de 7 colonnes :


<div>
    <table style="margin-left: 0px;">
        <thead>
            <tr>
                <th>Colonne</th>
                <th style="text-align: left">Description</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>anime_id</td>
                <td style="text-align: left">Identifiant unique de myanimelist.net qui identifie un anime</td>
            </tr>
            <tr>
                <td>name</td>
                <td style="text-align: left">Le nom de l'anime</td>
            </tr>
            <tr>
                <td>genre</td>
                <td style="text-align: left">La liste des genres d'un anime séparés par une vigurle</td>
            </tr>
            <tr>
                <td>type</td>
                <td style="text-align: left">movie, TV, OVA, etc.</td>
            </tr>
            <tr>
                <td>episodes</td>
                <td style="text-align: left">Le nombre d'épisodes (1 s'il s'agit d'un film)</td>
            </tr>
            <tr>
                <td>rating</td>
                <td style="text-align: left">La note moyenne sur 10 des notes de tous les utilisateurs</td>
            </tr>
            <tr>
                <td>members</td>
                <td style="text-align: left">Le nombre d'utilisateurs ayant ajouté l'anime à leur liste de visionnage</td>
            </tr>
        </tbody>
    </table>
</div>

### Rating Dataset
Ce dataset appelé **rating** contient **7813737** entrées de 3 colonnes :

<div>
    <table style="margin-left: 0px;">
        <thead>
            <tr>
                <th>Colonne</th>
                <th style="text-align: left">Description</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>user_id</td>
                <td style="text-align: left">Identifiant généré aléatoirement pour représenter un utilisateur</td>
            </tr>
            <tr>
                <td>anime_id</td>
                <td style="text-align: left">L'identifiant de l'anime noté par l'utilisateur</td>
            </tr>
            <tr>
                <td>rating</td>
                <td style="text-align: left">La note sur 10 attribué par l'utilisateur à l'anime (-1 si l'utilisateur n'a pas noté mais l'a ajouté à sa liste de visionnage)</td>
            </tr>
        </tbody>
    </table>
</div>

## Traitement du dataset

Nous allons commencer par prendre connaissance de nos données

In [ ]:
df_anime.head()

In [ ]:
df_anime.info()

In [ ]:
df_rating.head()

In [ ]:
df_rating.info()

### Traitement des valeurs nulles

In [ ]:
df_anime.isnull().sum().sort_values(ascending=False)

In [ ]:
df_rating.isnull().sum().sort_values(ascending=False)

Il manque quelques valeurs quelques valeurs dans notre liste d'anime.
Comme ces animes représentent au plus 1% du nombre total d'animes dans notre dataset, nous proposons d'effectuer les manipulations suivantes :
1. Drop des 230 animes sans note
2. Complétions des genres manquants par le genre le plus prominent dans le dataset
3. Complétion des types manquants par le type le plus prominent dans le dataset

In [ ]:
# On supprime les animes n'ayant pas de notes
df_anime = df_anime[~np.isnan(df_anime['rating'])]

# On remplace les genres manquants par le genre le plus prominent dans le dataset
print(f"Genre le plus prominent: {df_anime['genre'].mode().values[0]}")
df_anime['genre'] = df_anime['genre'].fillna(df_anime['genre'].dropna().mode().values[0])

# On remplace les types manquants par le type le plus prominent dans le dataset
print(f"Type le plus prominent: {df_anime['type'].mode().values[0]}")
df_anime['type'] = df_anime['type'].fillna(df_anime['type'].dropna().mode().values[0])

On vérifie que nous avons bien éliminé les données qui étaient **null** :

In [ ]:
df_anime.isnull().sum()

### Traitement sur le reste des données

Un utilisateur n'ayant pas donné de note à un anime est représenté par une note de -1, nous pouvons donc remplacer les -1 par des valeurs `NaN`.

In [ ]:
df_rating['rating'] = df_rating['rating'].apply(lambda x: np.nan if x==-1 else x)
df_rating.head()

Nous allons à présent créer un nouveau dataset en fusionnant nos deux premiers dataset de telle sorte à ce que :
1. Seuls les animes du type `TV` soient inclus dans notre dataset
2. Il ne reste que l'utilisateur, le nom de l'anime et la note attribuée dans le nouveau dataset

In [ ]:
df_anime = df_anime[df_anime['type']=='TV']

df_rated_animes = df_rating.merge(df_anime, left_on='anime_id', right_on='anime_id', suffixes=['_user', ''])

df_rated_animes = df_rated_animes[['user_id', 'name', 'rating']]

In [ ]:
df_rated_animes.head()

## Exploration des données

Avant d'aller plus loin vers la construction du modèle, nous allons explorer un peu le type de données avec lesquelles nous allons travailler :

### Top 10 des animes en fonction de leur score

In [ ]:
anime_rating_count = (df_rated_animes.groupby(by = ['name'])['rating'].count().reset_index()[['name', 'rating']])
top10_anime_rating = anime_rating_count[['name', 'rating']].sort_values(by = 'rating',ascending = False).head(10)

_, ax = plt.subplots(figsize=(15, 7))
fig = sns.barplot(x="name", y="rating", data=top10_anime_rating,  palette="Dark2", ax=ax)
fig.set_xticklabels(fig.get_xticklabels(), fontsize=11, rotation=40, ha="right")
fig.set_title('Top 10 des animes en fonction de leur score', fontsize = 22)
fig.set_xlabel('Anime', fontsize = 20)
fig.set_ylabel('Score utilisateur', fontsize = 20)

Si on compare le top 10 des animés les mieux notés au top 10 des animés par communauté (cad nombre d'utilisateurs membres) on se rend compte qu'effectivement les animes les mieux notés sont également ceux qui ont été les plus visionnés.

In [ ]:
top10_anime_members = df_anime[['name', 'members']].sort_values(by = 'members',ascending = False).head(10)

_, ax = plt.subplots(figsize=(15, 7))
fig = sns.barplot(x="name", y="members", data=top10_anime_members,  palette="gnuplot2", ax=ax)
fig.set_xticklabels(fig.get_xticklabels(), fontsize=11, rotation=40, ha="right")
fig.set_title('Top 10 des animes en fonction de leur nombre de membres', fontsize = 22)
fig.set_xlabel('Anime', fontsize = 20)
fig.set_ylabel('Membres', fontsize = 20)

### Distribution des notes

In [ ]:
plt.figure(figsize = (15, 7))
plt.subplot(1,2,1)
df_rated_animes['rating'].hist(bins=70)
plt.title("Notes attribuées par les utilisateurs")

Nous remarquons que la grande majorité des notes sont comprises entre 6 et 9.5

In [ ]:
df_rated_animes.rating.describe()

In [ ]:
df_rated_animes.rating.mode().values[0]

Sur notre échantillon complet, nous pouvons voir que les utilisateurs ont tendance à bien noté les animes qu'ils ont vu, avec une note moyenne de **7.73/10**.

### Exploration des genres les plus prominents parmi les animes restant

In [ ]:
from collections import defaultdict

all_genres = defaultdict(int)

for genres in df_anime['genre']:
    for genre in genres.split(','):
        all_genres[genre.strip()] += 1

from wordcloud import WordCloud

genres_cloud = WordCloud(width=800, height=400, background_color='white', colormap='gnuplot').generate_from_frequencies(all_genres)
plt.imshow(genres_cloud, interpolation='bilinear')
plt.axis('off')

Nous voyons donc que la pluspart des animes avec lesquels nous travaillons sont dans la catégorie **Comédie**, **Action** et **Aventure** !

## Système de recommandations

Maintenant que nous avons vu à peu près à quoi resemblait nos données, nous allons pouvoir commencer à créer notre système de recommandation.

### Principe de fonctionnement

Nous cherchons à construire un **système de racommendations** à l'aide d'une méthode appelé le **filtrage collaboratif**. Il existe trois type de système de filtrage qui dépend notamment des données que nous avons à dispoisition :
- Le filtrage collaboratif actif (Netflix)
- Le filtrage collaboratif passif (Facebook, Amazon)
- Le filtrage basé sur le contenu (recherche par image, par document)

[Page Wikipedia](https://fr.wikipedia.org/wiki/Filtrage_collaboratif)

Ici, nous sommes dans un cas de **filtrage collaboratif actif** car nous avons à disposition des données explicites de l'utilisateur concernant ses visionnages et ses goûts déclarés (à partir des notes données). Il s'agit notamment du type de filtrage utilisé par des sites comme Netflix pour recommander ses shows. La principale faiblesse de ce type de système étant que les données recueillies peuvent contenir un biais de "déclaration".

![Image](https://miro.medium.com/max/2656/1*6_NlX6CJYhtxzRM-t6ywkQ.png)

### Préparation des données

Nous allons calculer une table de pivot afin de pouvoir déterminer la similitude entre les différents animes.

In [ ]:
pivot = df_rated_animes.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot.head()

Nous allons à présent appliquer les transformations suivantes sur notre pivot :
1. Normalisation des valeurs
2. Remplacer les valeurs NaN par 0
3. Transposer notre table de pivot
4. Drop des colonnes avec des 0 (tous les animes qui n'ont pas étés notés)
5. On va enfin utiliser `scipy` pour convertir notre tableau en [matrice creuse](https://fr.wikipedia.org/wiki/Matrice_creuse) pour le calcul de similitude.

In [ ]:
# Etape 1
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# Etape 2
pivot_n.fillna(0, inplace=True)

# Etape 3
pivot_n = pivot_n.T

# Etape 4
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

# Etape 5
piv_creux = sp.sparse.csr_matrix(pivot_n.values)

# C'est prêt !
print("Pivot prêt")

## Création du modèle

Nous allons nous baser sur un modèle de **[similarité cosinus](https://fr.wikipedia.org/wiki/Similarit%C3%A9_cosinus)** pour trouver les animes qui se rapprochent le plus d'un anime donné à partir de notre table de pivot. Mathématiquement, nous allons comparer deux vecteurs à *n* dimensions afin d'en mesurer l'angle, plus cet angle est petit, plus ces deux vecteurs sont similaires.

Pour ce faire, nous aurons recours à la formule suivante fournie dans `sklearn` :
![Image](https://cdn-images-1.medium.com/max/579/1*5hJibEtQPavnbgRxg8w2Fg.gif)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

anime_similarity = cosine_similarity(piv_creux)

df_anime_sim = pd.DataFrame(anime_similarity, index=pivot_n.index, columns=pivot_n.index)

In [ ]:
def recommendation(name):
    num = 1
    print(f"Puisque vous avez vu {name} nous vous recommandons :\n")
    for anime in df_anime_sim.sort_values(by=name, ascending=False).index[1:6]:
        print(f"#{num}: {anime}, similaire à {round(df_anime_sim[anime][name]*100, 2)}%")
        num += 1

In [ ]:
recommendation("Death Note")

In [ ]:
recommendation("Toradora!")

## Déterminer la note d'un anime

Nous allons maintenant nous intéresser à la possibilité de déterminer la note moyenne d'un anime sur **MyAnimeList**.

A ce stade, notre dataframe `anime` ne contient plus que des animes de type "TV" dont les notes ne sont pas nulles. Nous proposons donc d'effectuer les changements suivants sur ce dataframe afin d'en faciliter l'exploitation pendant notre phase d'entraînement :
* Suppression des colonnes `anime_id`, `name` et `TV` qui sont soit des valeurs uniques soit une valeur constante et qui donc n'apporteront rien à l'entraînement.
* Séparation des différents genres en plusieures colonnes contenant un 1 ou un 0 en si l'anime est du genre X ou non.
* Ajout d'une composante "similarité aux animes du Top 10" qui sera le pourcentage moyen de similarité d'un anime X avec les autres animes du top 10 calculé précédemment.

On commence donc par supprimer nos premières colonnes :

In [ ]:
# On drop les colonnes 'type', 'anime_id' et nous dropperons la colonne 'name' plus tard car nous en aurons besoin pour les calculs de similarité
df_anime.drop(['anime_id', 'type'], axis=1, inplace=True)

Nous allons maintenant créer nos différentes colonnes de genres. La principale difficulté étant qu'un anime peut avoir plusieurs genres. Nous allons donc devoir séparer la liste de genres de chaque anime avant de pouvoir les ranger dans un tableau contenant un genre unique à chaque fois.

In [ ]:
# Les genres différents
all_genres = df_anime.genre.str.split(', ', expand=True).stack().unique()
all_genres

In [ ]:
# Nombre de genre différents
num_genres = len(all_genres)
num_genres

Pour créer les colonnes, nous allons mapper chaque genre par un 1 ou un 0 en convertissant le résultat de la fonction "contains" en entier. Enfin nous transformons notre résultat en tableau afin de pouvoir l'affecter directement à une de genre.

In [ ]:
# Création des colonnes de genre individuelles
for i in range(num_genres):
    df_anime[all_genres[i]] = df_anime.genre.str.contains(all_genres[i]).astype(int).to_numpy()

In [ ]:
# On peut maintenant drop la colonne 'genre'
df_anime.drop('genre', axis=1, inplace=True)

Maintenant, nous allons calculer la similarité moyenne entre chaque anime et les animes du top 10. Le raisonnement ici étant qu'un anime semblable à un anime déjà bien noté a de fortes chance d'être bien noté lui-même.

In [ ]:
def top10_sim_moy(name):
    # cas où l'anime n'est pas dans notre table de pivot
    if name not in df_anime_sim:
        return np.nan
    sim = 0
    for row in top10_anime_rating.name:
        sim += df_anime_sim[name][row]
    return 100*(round(sim/10, 3))

In [ ]:
print("Anime du top 10")
print(top10_sim_moy("Death Note"))
print(top10_sim_moy("Toradora!"))

print("Anime plutôt bien noté")
print(top10_sim_moy("Dragon Ball"))
print(top10_sim_moy("K-On!!"))

print("Anime plutôt mal noté")
print(top10_sim_moy("Asobo Toy-chan"))
print(top10_sim_moy("RGB Adventure"))

On remarque qu'avec cette méthode de notation, il semble plutôt difficile de distinguer un anime du top 10 d'un anime avec une note "acceptable". De plus il semble étrange qu'un anime du top 10 ne soit similaire qu'à ~40% des animes du top 10 puisqu'il en fait partie lui-même.

Nous proposons une méthode alternative où l'on retiendra la valeur maximale :

In [ ]:
def top10_sim_max(name):
    # cas où l'anime n'est pas dans notre table de pivot
    if name not in df_anime_sim:
        return np.nan
    sim = 0
    for row in top10_anime_rating.name:
        sim = max(sim, df_anime_sim[name][row])
    return 100*(round(sim, 3))

In [ ]:
print("Anime du top 10")
print(top10_sim_max("Death Note"))
print(top10_sim_max("Toradora!"))

print("Anime plutôt bien noté")
print(top10_sim_max("Dragon Ball"))
print(top10_sim_max("K-On!!"))

print("Anime plutôt mal noté")
print(top10_sim_max("Asobo Toy-chan"))
print(top10_sim_max("RGB Adventure"))

Les résultats semblent ainsi présenter une plus grande exploitabilité. Pour s'en assurer, nous allons construire deux tableau avec les valeurs de similarité pour chaque anime et nous observerons la distribution de ces valeurs.

In [ ]:
arr_moy = []
arr_max = []

for row in df_anime.name:
    arr_moy.append(top10_sim_moy(row))
    arr_max.append(top10_sim_max(row))

In [ ]:
df_sims = pd.DataFrame({'sim_moy': arr_moy, 'sim_max': arr_max})
df_sims.head()

In [ ]:
df_sims.describe()

In [ ]:
df_sims.mode()

A toute première vue, la similarité moyenne semble être la plus intéressante à utiliser puisque si l'on considère la similarité maximale, alors 50% des animes ont une similarité de 0% ce qui semble peu exploitable.

Nous allons tout de même tracer ces deux distributions pour en avoir le coeur net :

In [ ]:
plt.figure(figsize = (15, 7))
plt.subplot(1,2,1)
df_sims['sim_moy'].hist(bins=70)
plt.title("Similarité moyenne des animes avec ceux du Top10")

In [ ]:
plt.figure(figsize = (15, 7))
plt.subplot(1,2,1)
df_sims['sim_max'].hist(bins=70)
plt.title("Similarité maximale des animes avec ceux du Top10")

Nous voyons donc bien d'après ces distribution qu'utiliser la fonction de similarité maximale est moins performante car nous allons perdre l'information sur toutes les entrées qui ont une similarité inférieure à 0. Nous allons donc conserver la similarité moyenne.

Nous avons cependant quelques valeures nulles à traiter.

In [ ]:
df_sims.sim_moy.isnull().sum()

Nous allons générer 269 valeurs aléatoirement en suivant une loi normale à partir de la distribution existante des valeurs de notre fonction de similarité moyenne :

In [ ]:
mean = df_sims.sim_moy.mean()
std  = df_sims.sim_moy.std()

for i in range(df_sims.sim_moy.shape[0]):
    # On remplace les valeurs NaN
    if np.isnan(df_sims.loc[i, "sim_moy"]):
        df_sims.loc[i, "sim_moy"] = np.random.normal(mean, std)

df_sims.sim_moy.isnull().sum()

Maintenant que nous n'avons plus de valeures nulles, nous proposons d'effectuer les modifications suivantes sur notre distribution avant de l'associer à notre DataFrame principal :
* Mapping des données entre 0 et 1, ainsi un anime fortement similaire aux animes du top 10 aura un coefficient de similarité proche de 1 tandis qu'un différent aura un coefficient proche de 0.

In [ ]:
from sklearn import preprocessing

In [ ]:
min_max = preprocessing.MinMaxScaler()
df_sims["sim_moy"] = min_max.fit_transform(df_sims[["sim_moy"]])

df_sims.sim_moy.describe()

In [ ]:
plt.figure(figsize = (15, 7))
plt.subplot(1,2,1)
df_sims['sim_moy'].hist(bins=70)
plt.title("Similarité moyenne des animes avec ceux du Top10")

Nous pouvons maintenant ajouter notre coefficient de similarité à notre dataframe principal :

In [ ]:
df_anime["top10_sim"] = df_sims.sim_moy.to_numpy()

# On peut drop la colonne 'name' à présent
df_anime.drop('name', axis=1, inplace=True)

Nous obtenons alors pour finir la dataframe suivante :

In [ ]:
df_anime

In [ ]:
df_anime.info()

Nous remarquons cependant que la colonne `episodes` n'est pas un entier mais un type `object`, jetons un oeil :

In [ ]:
df_anime.episodes.unique()

Il nous faut encore transformer légèrement notre dataframe pour changer nos épisodes en entier et remplacer les valeurs Unknown de la même manière que nous avons remplacé les notes manquantes.

In [ ]:
# Les chaines non-transformables seront transformées en NaN
df_anime.episodes = pd.to_numeric(df_anime.episodes, errors="coerce", downcast="integer")

In [ ]:
# On remplace en suite les valeurs nulles
mean = df_anime.episodes.mean()
std  = df_anime.episodes.std()

df_anime.loc[np.isnan(df_anime.episodes), "episodes"] = np.random.normal(mean, std)

df_anime.episodes.isnull().sum()

Regardons la distribution de nos notes :

In [ ]:
plt.figure(figsize = (15, 7))
plt.subplot(1,2,1)
df_anime['rating'].hist(bins=70)
plt.title("Notes moyenne attribuées aux animes")

Nous avons des notes qui sont décallées vers la droite. Comme nous l'avions vu précédemment, les utilisateurs ont plutôt tendance à aimer les animes qu'ils regardent. Le principal problème est alors que ceci pourrait engendrer un biais dans notre modèle à l'avenir. Pour éviter ce problème, je propose de standardiser notre distribution :

In [ ]:
standard = preprocessing.StandardScaler()
df_anime["rating"] = standard.fit_transform(df_anime[["rating"]])

df_anime.rating.describe()

In [ ]:
plt.figure(figsize = (15, 7))
plt.subplot(1,2,1)
df_anime['rating'].hist(bins=70)
plt.title("Notes moyenne attribuées aux animes")

Jetons un oeil aux corrélations qui existent à présent dans notre dataframe

In [ ]:
corr = df_anime.corr()
fig, ax = plt.subplots(figsize=(24,20))
sns.heatmap(corr, ax=ax)

Nous sommes ici principalement intéressés par la composante `rating` puisqu'il s'agira de la composante que nous chercherons à déterminer avec notre algorithme de machine learning. Nous pouvons cependant faire quelques observations intéressantes :
* Comme nous puvions imaginer, le nombre de membres ainsi que la similarité d'un anime avec ceux du top 10 possède une corrélation positive.
* Nous remarquons aussi que le coefficient de similarité possède également une corrélation positive avec le nombre de membres. Ainsi, plus un anime est semblable à ceux du top 10, plus il possède de membre et donc, plus il a de chance d'avoir une bonne note.
* Le genre d'un anime semble ne pas influer énormément sur sa note à l'exception des genre du "Drama", "Romance", "Shounen", "Supernatural" et "Psychological" qui ont la corrélation positive la plus forte avec sa note moyenne et surtout le genre "kids" qui se voit sévèrement punni d'une corrélation négative : à comprendre que si un anime est classé pour enfants, ce dernier est plutôt mal noté.
* Une corrélation un peu hors-propos mais amusante : le genre "Sci-Fi" est fortement corrélé avec le genre "Mecha" (en même temps, avoir les robotos géants dans une oeuvre la classe souvent comme étant de la science fiction :))

Je propose d'observer les corrélations suivantes plus en détail : 
* Le rapport entre le nombre de membres d'un anime et sa note moyenne.
* Le rapport entre la similarité d'un anime avec ceux du top 10 et sa note moyenne.
* Le rapport entre la similarité d'un anime avec ceux du top 10 et son nombre de membres.

In [ ]:
sns.relplot(x='members', y='rating', data=df_anime, kind='line')

In [ ]:
sns.relplot(x='top10_sim', y='rating', data=df_anime, kind='line')

In [ ]:
sns.relplot(x='top10_sim', y='members', data=df_anime, kind='line')

A partir de ces graphs nous pouvons faire les observations suivantes :
* Nous voyons qu'effectivement plus un anime a de membres, plus il a de chance d'avoir une note élevée. Plus le nombre de membres est bas et plus la note devient chaotique
* Dans le cas de la similarité, nous remarquons que les notes sont plutôt stable sur les extrèmes (les animes les plus différents de ceux du top 10 et ceux les plus semblables sont ceux qui ont une note stable). Les notes deu centre étant de nouveau une forte variation.
* Avec le nombre de membres en fonction de la similarité, nous pouvons voir une observation intéressante : il semble y avoir deux groupes parmi les utilisateurs, à savoir ceux qui recherchent des animes similaires aux animes populaire (coeff de similarité entre 0.4 et 1) et ceux qui recherchent des animes plus "exotiques" (coeff de similarité entre 0 et 0.4). Ce qui explique pourquoi les notes se stabilisent pour notre deuxième grah sur les extrèmes : ce sont ceux les plus prisés par deux pans de la communauté.

## Régression

A présent, il est temps de construire nos modèles. Nous considérerons que nous sommes face à un problème de **régression** car nous tentons de donner une note à un anime. On aurait également pu faire le choix d'effectuer une *classification*, par exemple en arrondissant les notes des animes à l'entier le plus proche. Néanmoins cela revient à vouloir classer un anime dans une catégorie parmi dix existantes.

### Résultats obtenus (détaillé plus bas)

- **Régression Linéaire**: 45%
- **Régression Forêt aléatoire**: 63%
- **Régression XGBoost**: 56%
- **Régression HistGradientBoosting**: 64%

### Préparation des données d'entraînement

In [ ]:
# Import de nos librairies pour le modèle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Import des modèles
from sklearn import linear_model
from sklearn import ensemble
import xgboost as XGB

In [ ]:
# Préparation des données d'entraînement
X = df_anime.drop(['rating'], axis=1)
y = df_anime.rating
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

### Régression Lasso

In [ ]:
lm = linear_model.LinearRegression()
lm.fit(X_train, y_train)
y_lm = lm.predict(X_test)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_lm)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Note")
plt.ylabel("Prediction de note")
plt.title("Notes réelles vs prédictions")

In [ ]:
lm.score(X_test,y_test)

### Régression ElasticNet

In [ ]:
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Note")
plt.ylabel("Prediction de note")
plt.title("Notes réelles vs prédictions")

In [ ]:
rf.score(X_test,y_test)

In [ ]:
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Note")
plt.ylabel("Prediction de note")
plt.title("Notes réelles vs prédictions")

In [ ]:
xgb.score(X_test, y_test)

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting

In [ ]:
est = ensemble.HistGradientBoostingRegressor()
est.fit(X_train, y_train)
y_est = est.predict(X_test)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_est)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Note")
plt.ylabel("Prediction de note")
plt.title("Notes réelles vs prédictions")

In [ ]:
est.score(X_test, y_test)